In [1]:
import os

from openai import OpenAI

In [5]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
)

completion = client.chat.completions.create(
    model="openai/gpt-oss-20b:free",
    messages=[
        {
            "role": "user",
            "content": "こんにちわ！",
        }
    ],
)
print(completion.choices[0].message.content)

こんにちは！今日はどんなことをお手伝いしましょうか？
